<a href="https://colab.research.google.com/github/cppel/dz/blob/main/Task2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Нейросеть для California housing

Будем работать с набором данным `fetch_california_housing` из `sklearn`.

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd


data = fetch_california_housing()

X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = data['target']

X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


#### 1.1. Разбейте данные на обучение и тест

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.3, random_state=465)
X_train.shape, X_test.shape

((14448, 8), (6192, 8))

1.1.1 Нормализация данных по гит

In [ ]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

#### 1.2. Создайте архитектуру для этой задачи

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(128, input_shape=(8, ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))

#### 1.3. Скомпилируйте сеть

In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1152      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 9,473
Trainable params: 9,473
Non-trainable params: 0
_________________________________________________________________


1.4 Обучите нейросеть

In [ ]:
history = model.fit(X_train, y_train, epochs=100, validation_split=0.05)

Epoch 1/100
429/429 [==============================] - 3s 3ms/step - loss: 0.7290 - mae: 0.5883 - val_loss: 0.5971 - val_mae: 0.5133
Epoch 2/100
429/429 [==============================] - 1s 3ms/step - loss: 0.3958 - mae: 0.4457 - val_loss: 0.4840 - val_mae: 0.4457
Epoch 3/100
429/429 [==============================] - 1s 3ms/step - loss: 0.3853 - mae: 0.4317 - val_loss: 0.3996 - val_mae: 0.4088
Epoch 4/100
429/429 [==============================] - 1s 3ms/step - loss: 0.3525 - mae: 0.4202 - val_loss: 0.3666 - val_mae: 0.4092
Epoch 5/100
429/429 [==============================] - 1s 3ms/step - loss: 0.3414 - mae: 0.4103 - val_loss: 0.3612 - val_mae: 0.3943
Epoch 6/100
429/429 [==============================] - 1s 3ms/step - loss: 0.3243 - mae: 0.4008 - val_loss: 0.3222 - val_mae: 0.3872
Epoch 7/100
429/429 [==============================] - 2s 5ms/step - loss: 0.3140 - mae: 0.3919 - val_loss: 0.3333 - val_mae: 0.3901
Epoch 8/100
429/429 [==============================] - 1s 3ms/step - 

#### 1.5. Сделайте вывод, хорошо ли модель обучается?

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [ ]:
fig = go.Figure()
print(history.history.keys())
fig.add_trace(go.Scattergl(y=history.history['mae'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_mae'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Mean Absolute Error')

fig.show()

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


Метрика

In [ ]:
mse_nn, mae_nn = model.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

194/194 [==============================] - 0s 2ms/step - loss: 0.2902 - mae: 0.3647
Mean squared error on test data:  0.29024621844291687
Mean absolute error on test data:  0.364724338054657


#### 1.6. Сделайте предсказания на тестовых данных

In [ ]:
from sklearn.metrics import mean_absolute_error

Xnew = model.predict(X_test)

print(f"Средняя абсолютная ошибка: {int(mean_absolute_error(Xnew, y_test) * 100000)}$")

194/194 [==============================] - 0s 2ms/step
Средняя абсолютная ошибка: 36472$


Вывод

Исходя из полученной метрики можно сделать вывод, что погрешность в пределах нормы около 30 тысяч долларов. 

Попытка через нормализацию минмакс

Несколько вариаций

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd


data = fetch_california_housing()

X = pd.DataFrame(data['data'], columns=data['feature_names'])
y = data['target']

X.head()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y, test_size=0.1, random_state=111)
X_train.shape, X_test.shape

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(128, input_shape=(8, ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

history = model.fit(X_train, y_train, epochs=120, validation_split=0.1)



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 128)               1152      
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_output (Dense)        (None, 1)                 65        
                                                                 
Total params: 9,473
Trainable params: 9,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/120
523/523 [==============================] - 3s 3ms/step - loss: 0.7215 - mae: 0.5627 - val_loss: 0.4060 - val_mae: 0.4609
Epoch 2/120
523/523 [==============================] - 1s 2ms/step - loss: 0.3794 - mae: 0.4334 - val_loss: 0.3554 - val_mae: 0.4176
Epoch 3/120
523/523 [=============================

In [ ]:
import plotly.graph_objs as go
fig = go.Figure()
fig.add_trace(go.Scattergl(y=history.history['loss'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_loss'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [ ]:
fig = go.Figure()
print(history.history.keys())
fig.add_trace(go.Scattergl(y=history.history['mae'],
                    name='Train'))

fig.add_trace(go.Scattergl(y=history.history['val_mae'],
                    name='Valid'))


fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Mean Absolute Error')

fig.show()

mse_nn, mae_nn = model.evaluate(X_test, y_test)

print('Mean squared error on test data: ', mse_nn)
print('Mean absolute error on test data: ', mae_nn)

from sklearn.metrics import mean_absolute_error

Xnew = model.predict(X_test)

print(f"Средняя абсолютная ошибка: {int(mean_absolute_error(Xnew, y_test) * 100000)}$")

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])


65/65 [==============================] - 0s 2ms/step - loss: 0.3029 - mae: 0.3552
Mean squared error on test data:  0.3028726279735565
Mean absolute error on test data:  0.35523515939712524
65/65 [==============================] - 0s 2ms/step
Средняя абсолютная ошибка: 35523$


В целом результаты не сильно поменялись